In [3]:
from zipfile import ZipFile
import os

# Define the path to the uploaded zip file and the extraction directory
zip_file_path = '../../module7/data/Airline_review.csv.zip'

# Read the csv file inside the zip file without extracting the file into the file system
import pandas as pd
with ZipFile(zip_file_path, 'r') as zip_ref:
    with zip_ref.open('Airline_review.csv') as file:
        df = pd.read_csv(file)
        


In [5]:
import nltk

In [11]:
# Manually define a basic set of English stopwords
manual_stop_words = {
    'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 
    'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 
    'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 
    'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 
    'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 
    'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',
    'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 
    'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 
    'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 
    'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', 
    "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 
    'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', 
    "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"
}
import re

def clean(text):

    text = re.sub('[^a-zA-Z\s]', '', text).lower()
    # Tokenize the text
    words = nltk.word_tokenize(text)
    
    # Remove stopwords
    words = [x for x in words if x not in manual_stop_words]
    
    return words

reviews = df['Review'].apply(clean).tolist()

In [12]:
# use gensim to create a dictionary and a corpus
from gensim import corpora

dictionary = corpora.Dictionary(reviews)
corpus = [dictionary.doc2bow(review) for review in reviews]




In [17]:
# Train the LDA model
from gensim import models

num_topics = 10
lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, 
                            passes=5, alpha='auto', eta='auto',  random_state=10)

In [18]:

# Print the topics
for i in range(lda_model.num_topics):
    print (str(i) + ':' + ' '.join([w[0] for w in lda_model.show_topic(i)]))

# Assign the topics to the reviews
topics = lda_model.get_document_topics(corpus)


0:flight good food crew service cabin time seats friendly staff
1:luggage bag baggage pay kg extra carry bags checked wow
2:de la que volotea el en las con por un
3:airline air flights airlines service fly time like flying low
4:seat seats extra flight row paid get front plane back
5:luggage baggage lost bag bags claim days manila arrived suitcase
6:ticket pay check service online customer airline website said told
7:johannesburg china kuala lumpur test town cape klm chinese shanghai
8:flight us plane airport boarding staff time gate passengers minutes
9:flight airline hours get us service customer day time would


In [ ]:
import pyLDAvis.gensim as gensimvis
vis_data = gensimvis.prepare(lda_model, corpus, dictionary, sort_topics=False)
pyLDAvis.display(vis_data)